# **Stanford Transportation - ChargePoint EV Chargers - Sep-2023**

## **Code Utilities**

In [77]:
from datetime import datetime, timedelta
from functools import partial
from pathlib import Path
from pprint import pprint, pformat
from types import SimpleNamespace
from zoneinfo import ZoneInfo

import IPython.display
from IPython.display import display
import pandas
from schema import Schema, And, Use, Or
import yaml

# Schemas
## Existing file
class PathPrefixPrepender:
    def validate(self, data, path_prefix: Path = Path()):
        return path_prefix / data

existing_file_path_schema = Schema(And(Use(Path), PathPrefixPrepender(), Path.is_file))

## Dataset manifest
dataset_manifest_schema_dict = {}
dataset_manifest_schema = Schema(dataset_manifest_schema_dict)
dataset_manifest_schema_dict.update({str: Or(existing_file_path_schema, dataset_manifest_schema)})

## Easy access of schemas through namespace
schemas = SimpleNamespace(
    dataset_manifest = dataset_manifest_schema
)

# Dataset wrapper class
class Dataset:
    '''Convenience wrapper for datasets described by manifest.yml'''
    def __init__(self, manifest_path: Path | str) -> None:
        if isinstance(manifest_path, str):
            manifest_path = Path(manifest_path)

        # Open manifest
        with open(manifest_path) as f:
            manifest = yaml.safe_load(f)

        # Validate manifest
        manifest = schemas.dataset_manifest.validate(manifest, path_prefix = manifest_path.parent)

        # Create namespaces attributes for easy access and pandas dataframes from CSVs
        def dict_to_namespace(d: dict):
            for k, v in d.items():
                if isinstance(v, dict):
                    d[k] = dict_to_namespace(v)
                elif isinstance(v, Path) and v.suffix.lower() == ".csv":
                    d[k] = pandas.read_csv(v, keep_default_na=False)
            return SimpleNamespace(**d)
        
        self.__dict__.update(dict_to_namespace(manifest).__dict__)

# Functions
print_md = partial(IPython.display.display_markdown, raw = True)

# Output formatting
pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', 100)

# Timezone abbreviation to zoneinfo
tz_abbrev_to_zoneinfo = {
    "PDT": ZoneInfo("US/Pacific"),
    "PST": ZoneInfo("US/Pacific"),
}

# Convert alarm times to proper datetimes... why is this so hard???
def alarm_time_to_datetime(alarm_time):
    if isinstance(alarm_time, datetime):
        return alarm_time

    components = alarm_time.split()
    naive_time = datetime.strptime(" ".join(components[:-1]), "%Y-%m-%d %H:%M:%S")
    aware_time = naive_time.astimezone(tz_abbrev_to_zoneinfo[components[-1][1:-1]])
    return aware_time

## **Load Dataset**

In [78]:
dataset = Dataset("../data/manifest.yml")
station_overview = dataset.stations.overview
alarms = dataset.reports.alarms
# Convert alarm times to proper datetimes
alarms["Alarm Time"] = pandas.to_datetime(alarms["Alarm Time"].apply(alarm_time_to_datetime))
station_inventory = dataset.organizations.inventory.stations

## **Analysis**

### **Station Overview**

In [79]:
print_md("#### **General Information**")
print_md(f"**Number of Rows:** {len(station_overview)}")
print_md(f"**Columns:** {', '.join(station_overview.columns)}")

print_md("#### **Unique Column Values**")
for column in station_overview.columns:
    unique_column_values = ", ".join(filter(None, map(str, list(station_overview[column].unique()))))
    print_md(f"##### **{column.strip()}**")
    print_md(unique_column_values)

#### **General Information**

**Number of Rows:** 109

**Columns:** Model Number, Station Name, Org Name, MAC Address, System S/N, Address 1, Address 2, Floor Label, City, State, Zip/Postal Code, County, Country , Activation Status, Network Status, Station Status, Port 1 Status, Port 2 Status, No. of Ports, Pricing Policy Name, Reservations, Station Message, Enabled, Station Activation Type, Usable By, Visibility (Access Policy Name), EVSE ID, Radio Group Name, Software Version, Customer Category, Customer Subcategory, Circuit Sharing, Currency Name, Power Select / AC Breaker Rating, Scheduled Charging Policy Name, Warranty, Station Activation Date, Warranty Expiration Date, Warranty Service, Plug Type, Device Access Restriction, Waitlist Policy Name, Asset Tag ID, Meter ID, Service Delivery Point ID, Point of Service ID, Account ID, Circuit ID, Panel ID, Customer Name, Customer ID, Site ID, Site mode, Site Validation Status, Paired, Latitude, Longitude

#### **Unique Column Values**

##### **Model Number**

...

No

##### **Latitude**

37.426793659810016, 37.42680510868808, 37.42783416929919, 37.426802179905486, 37.4268107, 37.426785139713566, 37.4278075443645, 37.4278757041782, 37.43220734102387, 37.432241418935455, 37.45041, 37.43171044847697, 37.450394, 37.43175282649997, 37.45039733830973, 37.431757357410085, 37.45037, 37.43174023441741, 37.43175217797956, 37.4503876418016, 37.43187166339748, 37.450404144282025, 37.4235392953854, 37.43193981951077, 37.450413, 37.450653, 37.450396159211294, 37.42353031813845, 37.43199093655503, 37.42621544618659, 37.42925895023151, 37.42546342823224, 37.42563276795673, 37.42566152372096, 37.42562249843807, 37.42558910177513, 37.42552093987972, 37.425431477297856, 37.425595491949615, 37.43758993277954, 37.43994537522589, 37.437244799999995, 37.44008806162375, 37.440014588811074, 37.43996028277272, 37.483830085831535, 37.48380241662368, 37.48377900420903, 37.483708766921104, 37.483685354477096, 37.48385881922885, 37.4837598485916, 37.48374069296927, 37.483682161870526, 37.48382928768144, 37.4836513, 37.4837279225516, 37.483678969263806, 37.48382582903098, 37.483760912792704, 37.48375239918352, 37.48371408793006, 37.48384553997287, 37.48387691062148, 37.483716216333534, 37.483693069942404, 37.483680565567184, 37.48365555681048, 37.48380454502468, 37.483723665745266, 37.48365412633115, 37.48465735481143, 37.4846431782048, 37.4849383, 37.48494042836861, 37.484929786524965, 37.426269408993285, 37.426286449302594, 37.42632052990961, 37.426363130646514, 37.42638869107704, 37.42643129177518, 37.426439811911905, 37.426329050058925, 37.42631200975929, 37.42620976788015, 37.42622680820304, 37.426448332047634, 37.42632368474175, 37.42649998495956, 37.426465904434274, 37.426435544044864, 37.4264255395908, 37.42678556649676, 37.42639846349808, 37.426411897405714, 37.42634876198768, 37.426527734861885, 37.42647208247871, 37.426317033400885, 37.426334040085905

##### **Longitude**

-122.1592395153442, -122.15920263497024, -122.15657205847889, -122.15917514232784, -122.1591858711639, -122.1591322269836, -122.15916441349178, -122.15919659999996, -122.15915368465572, -122.15660022167354, -122.15649293331296, -122.16329689629517, -122.1632325232788, -122.177246, -122.1828376358759, -122.17716, -122.18284779288138, -122.17725536441804, -122.18285127986668, -122.17628, -122.18283775917509, -122.17633, -122.18286498364182, -122.17729425644876, -122.1826917650818, -122.17726810491084, -122.17110856613158, -122.18259520555728, -122.17723, -122.17668, -122.17727013988112, -122.17105130926204, -122.18247718836062, -122.17725, -122.17528922540896, -122.1844759464882, -122.15975492268448, -122.159489383992, -122.15954705148584, -122.15954674077344, -122.15958057909852, -122.15980588465575, -122.15982197790986, -122.15962885886078, -122.1714015910874, -122.17132815833588, -122.1716781, -122.1711511325409, -122.17118331904908, -122.17122623439332, -122.20392049695892, -122.20392586137694, -122.20395000125808, -122.20398486997529, -122.20401705648344, -122.20390306260032, -122.20396877672118, -122.20397682334824, -122.2040090098564, -122.20393357272786, -122.2037542, -122.2039915754978, -122.2040251031105, -122.2039164736454, -122.20396475340766, -122.20398755218427, -122.20399425770682, -122.20391669583968, -122.2038842871372, -122.2040063276474, -122.2040200739686, -122.20403113808078, -122.20404253746908, -122.20393122579496, -122.20393927242202, -122.20398218776626, -122.20403712440223, -122.20684677486878, -122.20742303177298, -122.2067073, -122.20677167301636, -122.20657855396728, -122.17548427354126, -122.17554864655762, -122.17566666375428, -122.17574176560667, -122.1758168674591, -122.17590269814758, -122.1759778, -122.17605290185242, -122.17551646004944, -122.17561301957396, -122.17535552750854, -122.17538771401672, -122.17614946137697, -122.17543473729552, -122.1761681952454, -122.17600726270445, -122.17591751852416, -122.17584955714722, -122.17600974127812, -122.17572525714723, -122.17569146903384, -122.17561776455688, -122.17609522209004, -122.17608156085204, -122.17560433068849, -122.17557698584444

### **Station and Port Status**

In [80]:
num_stations = len(station_overview)
num_ports = station_overview["No. of Ports"].sum()

print_md(f"**Number of Stations:** {num_stations}")
print_md(f"**Number of Ports:** {num_ports}")

print_md("#### **Station Count by Status**")
station_status_counts = station_overview.value_counts("Station Status")
assert num_stations == station_status_counts.sum()
display(station_status_counts.to_frame())

num_stations_need_service_or_watch_list = station_status_counts.loc[["Watch List", "Needs Service"]].sum()
percent_stations_need_service_or_watch_list = round(num_stations_need_service_or_watch_list / num_stations * 100, 2)
print_md(f"**Stations Needing Service or on Watch List:** {num_stations_need_service_or_watch_list} ({percent_stations_need_service_or_watch_list}%)")

print_md("#### **Port Count by Status**")
port_status_counts = station_overview[["Port 1 Status", "Port 2 Status"]].melt()["value"].value_counts().drop("")
assert num_ports == port_status_counts.sum()
display(port_status_counts.to_frame())

num_ports_need_service_or_watch_list = port_status_counts.loc[["Watch List", "Needs Service"]].sum()
percent_ports_need_service_or_watch_list = round(num_ports_need_service_or_watch_list / num_ports * 100, 2)
print_md(f"**Ports Needing Service or on Watch List:** {num_ports_need_service_or_watch_list} ({percent_ports_need_service_or_watch_list}%)")

**Number of Stations:** 109

**Number of Ports:** 205

#### **Station Count by Status**

,count
Station Status,
Available,88
Watch List,11
Needs Service,7
Inuse,3


**Stations Needing Service or on Watch List:** 18 (16.51%)

#### **Port Count by Status**

,count
value,
Available,144
In Use,31
Watch List,22
Needs Service,8


**Ports Needing Service or on Watch List:** 30 (14.63%)

### **Alarms**

In [81]:
print_md("#### **Troubled Stations (Watch List or Needs Service)**")
troubled_stations = station_overview[station_overview["Station Status"].isin(["Watch List", "Needs Service"])]
display(troubled_stations)

print_md("#### **Alarms From Troubled Stations (Last 30 days)**")
thirty_days_ago = pandas.to_datetime(datetime.now() - timedelta(days=30), utc=True)
month_alarms = alarms[alarms["Alarm Time"] > thirty_days_ago]
troubled_month_alarms = month_alarms[month_alarms["MAC Address"].isin(troubled_stations["MAC Address"])]
display(troubled_month_alarms)

print_md("#### **Alarm Name Counts and Frequency From Troubled Stations (Last 30 days)**")
alarm_name_counts_troubled = troubled_month_alarms.value_counts("Alarm Name")
alarm_name_freq_troubled = troubled_month_alarms.value_counts("Alarm Name", normalize=True)
display(pandas.concat([alarm_name_counts_troubled, alarm_name_freq_troubled], axis=1))

print_md("#### **Alarm Name Counts From Non-Troubled Stations (Last 30 days)**")
non_troubled_month_alarms = month_alarms[month_alarms["MAC Address"].isin(troubled_stations["MAC Address"]) == False]
alarm_name_counts_non_troubled = non_troubled_month_alarms.value_counts("Alarm Name")
alarm_name_freq_non_troubled = non_troubled_month_alarms.value_counts("Alarm Name", normalize=True)
display(pandas.concat([alarm_name_counts_non_troubled, alarm_name_freq_non_troubled], axis=1))

print_md("#### **Alarm Name Frequency of Troubled vs. Non-Troubled Stations (Last 30 days)**")
alarm_name_freq_compare = pandas.concat([alarm_name_freq_troubled, alarm_name_freq_non_troubled], axis=1)
alarm_name_freq_compare.columns = ["Troubled", "Non-Troubled"]
display(alarm_name_freq_compare)

#### **Troubled Stations (Watch List or Needs Service)**

,Model Number,Station Name,Org Name,MAC Address,System S/N,Address 1,Address 2,Floor Label,City,State,Zip/Postal Code,County,Country,Activation Status,Network Status,Station Status,Port 1 Status,Port 2 Status,No. of Ports,Pricing Policy Name,Reservations,Station Message,Enabled,Station Activation Type,Usable By,Visibility (Access Policy Name),EVSE ID,Radio Group Name,Software Version,Customer Category,Customer Subcategory,Circuit Sharing,Currency Name,Power Select / AC Breaker Rating,Scheduled Charging Policy Name,Warranty,Station Activation Date,Warranty Expiration Date,Warranty Service,Plug Type,Device Access Restriction,Waitlist Policy Name,Asset Tag ID,Meter ID,Service Delivery Point ID,Point of Service ID,Account ID,Circuit ID,Panel ID,Customer Name,Customer ID,Site ID,Site mode,Site Validation Status,Paired,Latitude,Longitude
0,CT4020-HD,STANFORD / THO NGW-B5,Stanford University,0024:B100:0003:12F8,184141024581,740 Serra St,,,Stanford,California,94305,Santa Clara County,United States,Activated,Unreachable,Watch List,Watch List,Watch List,2,$1 per hr enforcement hours,Disabled,Plugged In Required,Yes,Public,All Drivers,No Drivers,1942901,RG-GW1,4.6.0.46,Education,University / College,Disabled,U.S. Dollars (USD),,,ChargePoint Assure,2020-09-17 15:38:10,2023-12-31,Parts & Labor,J1772,NONE,,,,,,,,,,,,,Validated,No,37.426794,-122.159240
1,CT4020-HD,STANFORD / THO NGW-B4,Stanford University,0024:B100:0003:130F,184141024600,740 Serra St,,,Stanford,California,94305,Santa Clara County,United States,Activated,Unreachable,Watch List,Watch List,Watch List,2,$1 per hr enforcement hours,Disabled,Plugged In Required,Yes,Public,All Drivers,No Drivers,1942891,RG-GW1,4.6.0.46,Education,University / College,Disabled,U.S. Dollars (USD),,,ChargePoint Assure,2020-09-17 15:35:51,2023-12-31,Parts & Labor,J1772,NONE,,,,,,,,,,,,,Validated,No,37.426805,-122.159203
2,CT4020-HD,STANFORD / THO NGW-B3,Stanford University,0024:B100:0003:12EE,184141024577,740 Serra St,,,Stanford,California,94305,Santa Clara County,United States,Activated,Unreachable,Watch List,Watch List,Watch List,2,$1 per hr enforcement hours,Disabled,Plugged In Required,Yes,Public,All Drivers,No Drivers,1942871,RG-GW1,4.6.0.46,Education,University / College,Disabled,U.S. Dollars (USD),,,ChargePoint Assure,2020-09-17 15:29:31,2023-12-31,Parts & Labor,J1772,NONE,,,,,,,,,,,,,Validated,No,37.427834,-122.156572
3,CT4020-HD,STANFORD / THO NGW-B2,Stanford University,0024:B100:0003:12F1,184141024593,740 Serra St,,,Stanford,California,94305,Santa Clara County,United States,Activated,Unreachable,Watch List,Watch List,Watch List,2,$1 per hr enforcement hours,Disabled,Plugged In Required,Yes,Public,All Drivers,No Drivers,1942831,RG-GW1,4.6.0.46,Education,University / College,Disabled,U.S. Dollars (USD),,,ChargePoint Assure,2020-09-17 15:36:26,2023-12-31,Parts & Labor,J1772,NONE,,,,,,,,,,,,,Validated,No,37.426802,-122.159175
4,CT4020-HD,STANFORD / THO NGW-B1,Stanford University,0024:B100:0003:12D9,184141024567,740 Serra St,,,Stanford,California,94305,,United States,Activated,Unreachable,Watch List,Watch List,Watch List,2,$1 per hr enforcement hours,Disabled,Plugged In Required,Yes,Public,All Drivers,No Drivers,1942771,RG-GW1,4.6.0.46,Education,University / College,Disabled,U.S. Dollars (USD),,,ChargePoint Assure,2020-09-17 15:38:46,2023-12-31,Parts & Labor,J1772,NONE,,,,,,,,,,,,,Validated,No,37.426811,-122.159186
5,CT4020-HD,STANFORD / THO NGW-A5 ADA,Stanford University,0024:B100:0003:1318,184141024610,740 Serra St,,,Stanford,California,94305,Santa Clara County,United States,Activated,Unreachable,Watch List,Watch List,Watch List,2,$1 per hr enforcement hours,Disabled,Plugged In Required,Yes,Public,All Drivers,No Drivers,1943001,RG-GW2,4.6.0.46,Education,University / College,Disabled,U.S. Dollars (USD),,,ChargePoint Assure,2020-09-17 15:40:18,2023-12-31,Parts & Labor,J1772,NONE,,,,,,,,,,,,,Validated,No,37.426802,-122.159132
7,CT4020-HD,STANFORD / THO NGW-A3,Stanford University,0024:B100:0003:130E,

#### **Alarms From Troubled Stations (Last 30 days)**

,Display Name,MAC Address,Org Name,Alarm Severity,Alarm Name,Alarm ID,FRU Name,Alarm Time,Model Number,Port,Reason For Clearing,Component Serial Number
39,ROBLE / ROBLE LL1-6,0024:B100:0002:2954,Stanford University,,Unreachable,,,2023-09-26 11:54:26-07:00,CT4020-HD,,---,
58,ROBLE / ROBLE LL1-6,0024:B100:0002:2954,Stanford University,,Reachable,,,2023-09-26 11:23:41-07:00,CT4020-HD,,---,
76,ROBLE / ROBLE LL1-6,0024:B100:0002:2954,Stanford University,,Unreachable,,,2023-09-26 10:42:27-07:00,CT4020-HD,,---,
90,ROBLE / ROBLE LL1-6,0024:B100:0002:2954,Stanford University,,Circuit Sharing Current Restored,,,2023-09-26 09:42:50-07:00,CT4020-HD,2,---,
145,STANFORD / THO NGW-B5,0024:B100:0003:12F8,Stanford University,,Unreachable,,,2023-09-26 01:04:27-07:00,CT4020-HD,,---,
...,...,...,...,...,...,...,...,...,...,...,...,...
8948,STANFORD / THO NGW-A3,0024:B100:0003:130E,Stanford University,,Unreachable,,,2023-08-31 17:25:05-07:00,CT4020-HD,,---,
9011,STANFORD / STATION 5,0024:B100:0006:BD20,Stanford University,,Reachable,,,2023-08-31 14:55:38-07:00,CT4020-HD-GW,,---,
9028,STANFORD / STATION 5,0024:B100:0006:BD20,Stanford University,,IP Mismatch Detected,,,2023-08-31 14:45:38-07:00,CT4020-HD-GW,,---,
9029,STANFORD / STATION 5,0024:B100:0006:BD20,Stanford University,,Reachable,,,2023-08-31 14:45:38-07:00,CT4020-HD-GW,,---,


#### **Alarm Name Counts and Frequency From Troubled Stations (Last 30 days)**

,count,proportion
Alarm Name,,
Reachable,475,0.501584
Unreachable,292,0.308342
Fault Cleared,85,0.089757
Boot up,34,0.035903
Bootup Due to WATCHDOG,11,0.011616
Maintenance Required,11,0.011616
Circuit Sharing Current Restored,10,0.010560
Data Partition Full,9,0.009504
Bootup Due to SOFT RESET,7,0.007392


#### **Alarm Name Counts From Non-Troubled Stations (Last 30 days)**

,count,proportion
Alarm Name,,
Reachable,3188,0.393532
Unreachable,1929,0.238119
Circuit Sharing Current Restored,767,0.094680
Tamper Detect,761,0.093939
Fault Cleared,440,0.054314
Circuit Sharing Current Reduced,434,0.053574
IP Mismatch Detected,150,0.018516
Data Partition Full,126,0.015554
Bootup Due to SOFT RESET,114,0.014072


#### **Alarm Name Frequency of Troubled vs. Non-Troubled Stations (Last 30 days)**

,Troubled,Non-Troubled
Alarm Name,,
Reachable,0.501584,0.393532
Unreachable,0.308342,0.238119
Fault Cleared,0.089757,0.054314
Boot up,0.035903,0.011233
Bootup Due to WATCHDOG,0.011616,0.007530
Maintenance Required,0.011616,0.000741
Circuit Sharing Current Restored,0.010560,0.094680
Data Partition Full,0.009504,0.015554
Bootup Due to SOFT RESET,0.007392,0.014072
